## Preprocessing

In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import ssl
import certifi

# Set up SSL context correctly
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Load the dataset from the provided cloud URL
csv_url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"

#Issues loadind in csv - try/execept condition function
try:
    
    import urllib.request
    with urllib.request.urlopen(csv_url, context=ssl_context) as response:
        application_df = pd.read_csv(response)

    print("CSV file loaded successfully!")
    print(application_df.head())  
except Exception as e:
    print(f"Error loading CSV file: {e}")


CSV file loaded successfully!
        EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0             

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [6]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()
print(application_df["APPLICATION_TYPE"].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()


In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
print(application_df["CLASSIFICATION"].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
print(application_df)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                   False   
1           1    108590              1                   False   
2           1      5000              0                   False   
3           1      6692              1                   False   
4           1    142590              1                   False   
...       ...       ...            ...                     ...   
34294       1      5000              0                   False   
34295       1      5000              0                   False   
34296       1      5000              0                   False   
34297       1      5000              1                   False   
34298       1  36500179              0                   False   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                      True                 False                False   
1                     False                 False          

In [16]:
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values  # Drop target column
Y = application_df["IS_SUCCESSFUL"].values  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()


nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))



nn.add(tf.keras.layers.Dense(units=30, activation="relu"))



nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


nn.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=2)


Epoch 1/50
858/858 - 2s - 3ms/step - accuracy: 0.7235 - loss: 0.5690
Epoch 2/50
858/858 - 1s - 1ms/step - accuracy: 0.7296 - loss: 0.5555
Epoch 3/50
858/858 - 1s - 1ms/step - accuracy: 0.7315 - loss: 0.5518
Epoch 4/50
858/858 - 1s - 2ms/step - accuracy: 0.7315 - loss: 0.5503
Epoch 5/50
858/858 - 1s - 1ms/step - accuracy: 0.7317 - loss: 0.5488
Epoch 6/50
858/858 - 1s - 1ms/step - accuracy: 0.7323 - loss: 0.5484
Epoch 7/50
858/858 - 1s - 1ms/step - accuracy: 0.7329 - loss: 0.5464
Epoch 8/50
858/858 - 1s - 1ms/step - accuracy: 0.7339 - loss: 0.5465
Epoch 9/50
858/858 - 2s - 2ms/step - accuracy: 0.7333 - loss: 0.5460
Epoch 10/50
858/858 - 1s - 1ms/step - accuracy: 0.7335 - loss: 0.5455
Epoch 11/50
858/858 - 1s - 1ms/step - accuracy: 0.7341 - loss: 0.5449
Epoch 12/50
858/858 - 1s - 1ms/step - accuracy: 0.7351 - loss: 0.5449
Epoch 13/50
858/858 - 1s - 2ms/step - accuracy: 0.7341 - loss: 0.5441
Epoch 14/50
858/858 - 1s - 1ms/step - accuracy: 0.7344 - loss: 0.5435
Epoch 15/50
858/858 - 1s - 1m

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7254 - loss: 0.5590
Loss: 0.5590031743049622, Accuracy: 0.7253644466400146


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

nn.save("/Users/christinaland/Challenge 21/AlphabetSoupCharity.h5")
